# Morphometrics

In [1]:
import numpy as np
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

# Helper functions

In [2]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

def get_bearings(G, ID):
    # calculate the edge bearings
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    
    weight_by_length = False

    bearings = {}
    if weight_by_length:
        # weight bearings by length (meters)
        city_bearings = []
        for u, v, k, d in Gu.edges(keys=True, data=True):
            city_bearings.extend([d['bearing']] * int(d['length']))
        b = pd.Series(city_bearings)
        bearings[ID] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    else:
        # don't weight bearings, just take one value per street segment
        b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
        bearings[ID] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    return bearings[ID]

def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

def get_orientation_order(count):  
    try:
        H0 = 0
        for i in range(len(count)):
            Pi = (count[i]/sum(count))
            if Pi != 0:
                H0 += Pi*np.log(Pi)

        H0 = -1*H0
        Hmax = np.log(len(count))
        Hg = np.log(2)

        orientation_order = (1 - (((H0-Hg)/(Hmax-Hg))**2))
   
    except Exception as e:
        print(f'Error in orientation order {e}')

    return orientation_order

##### Functions to calculate compactness


def pp_compactness(geom): # Polsby-Popper
    p = geom.length
    a = geom.area    
    return (4*pi*a)/(p*p)

##### Functions to calculate fractal dimension

def fractal_dimension(Z, threshold=0.8):
    """Returns box-counting dimension of a 2D array.
    Args:
        Z: 2D array to be analysed.
        threshold: Cutoff for converting values in Z to 1 and 0.
    Returns:
        The estimated box counting dimension.
    """
    # Only for 2d image
    assert(len(Z.shape) == 2)
    def boxcount(Z, k):
        S = np.add.reduceat(np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0), np.arange(0, Z.shape[1], k), axis=1)
        # We count non-empty (0) and non-full boxes (k*k)
        return len(np.where((S > 0) & (S < k*k))[0])
    # Transform Z into a binary array
    Z = (Z < threshold)
    # Minimal dimension of image
    p = min(Z.shape)
    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))
    # Extract the exponent
    n = int(np.log(n)/np.log(2))
    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 1, -1)
    # Actual box counting with decreasing size
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))
    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]

##### Function to clean building heights

def clean_heights(x):
    try:
        return float(x)
    except ValueError:
        return 0

# Choose city

In [3]:
city_list = [
    'Melbourne',
    'Jerusalem',
    'Buenos Aires',
    'Paris',
    'Rotterdam',
    'Nashville',
    'Singapore',
    'Cape Town',
    'New York',
    'Los Angeles',
    'Chicago',
    'Boston',
    'Austin',
    'Seattle',
    'Philadelphia',
    'Pittsburgh',
    'Washington DC',
    'San Francisco',
    'SF Bay Area ',
    'Raleigh',
    'Milwaukee',
    'Portland',
    'San Diego',
    'Denver',
    'Miami',
    'Saint Louis',
    'Houston',
    'Atlanta',
    'Phoenix',
    'Detroit',
    'Minneapolis',
    'Savannah',
    'Charlotte',
    'Las Vegas',
    'Cincinnati',
    'Kansas City',
    'Nashville']


city = city_list[9]
print("City:", city)

data_folder = Path("../data")
input_file = data_folder / "0_boundaries" / (city + ".gpkg")
print("Reading:", input_file)
gdf = gpd.read_file(input_file, driver='GPKG')

City: Los Angeles
Reading: ../data/admin-levels/Los Angeles.gpkg


# Clean data

In [4]:
gdf['Center_point'] = gdf['geometry'].centroid
#Extract lat and lon from the centerpoint

gdf['lon'] = gdf.Center_point.map(lambda p: p.x)
gdf['lat'] = gdf.Center_point.map(lambda p: p.y)

gdf = gdf.drop(['Center_point'], axis = 1)

# Ensure crs is correct

gdf = ox.project_gdf(gdf,to_crs='epsg:4326',to_latlong=False) 

# Calculate area of every shape

temp = gdf.copy()
temp = temp.to_crs({'init': 'epsg:32630'})
temp["area_m^2"] = temp['geometry'].area
gdf["area_m^2"] = temp["area_m^2"]

# Define variables

In [5]:
# Scale Complexity
gdf['fractal-dimension'] = np.nan
gdf['compactness-area'] = np.nan
gdf['diameter-periphery'] = np.nan

# Spatial Complexity and Connectivity
gdf['shannon_entropy-street_orientation_order'] = np.nan
gdf['avg_streets_per_node'] = np.nan
gdf['avg_proportion_streets_per_node'] = np.nan
gdf['avg_street_length'] = np.nan
gdf['intersection_density'] = np.nan
gdf['street_density'] = np.nan
gdf['avg_circuity'] = np.nan
gdf['avg_node_connectivity'] = np.nan
gdf['avg_PageRank'] = np.nan
gdf['avg_betweenness_centrality'] = np.nan
gdf['avg_local_closeness_centrality'] = np.nan
gdf['avg_global_closeness_centrality'] = np.nan
gdf['avg_straightness_centrality'] = np.nan

# Built Complexity/Morphology
gdf['avg_building_area'] = np.nan
gdf['avg_tesselation_area'] = np.nan
gdf['avg_building_height'] = np.nan
gdf['avg_building_volume'] = np.nan
gdf['avg_building_orientation'] = np.nan
gdf['avg_tessellation_orientation'] = np.nan
gdf['avg_building_cell_alignment'] = np.nan
gdf['avg_street_alignment'] = np.nan
gdf['avg_width-street_profile'] = np.nan
gdf['avg_width_deviations-street_profile'] = np.nan
gdf['avg_openness-street_profile'] = np.nan
gdf['avg_heights-street_profile'] = np.nan
gdf['avg_heights_deviations-street_profile'] = np.nan
gdf['avg_profile-street_profile'] = np.nan
gdf['avg_building_compactness'] = np.nan

# Infrastructure variables/KPIs
gdf['total_area'] = np.nan
gdf['total_built_area'] = np.nan
gdf['total_street_length'] = np.nan

# Main loop

In [6]:
last_ID = 0
bookmark = True

# iterate through boundaries    
for ID in gdf.index: 
    
    # Pick up where we left off or the loop broke
    if (ID != last_ID) and (bookmark == True):
        continue
    else:
        bookmark = False
    
    print(f"Run {ID+1} out of {len(gdf)}: ID = {ID}")
    
    try:
        # get primary geometry and load network
        polygon = gdf.loc[ID,'geometry']
        G = ox.graph_from_polygon(polygon, network_type='drive', simplify=True, retain_all=False, truncate_by_edge=True, clean_periphery=True, custom_filter=None)
        
    except:
        pass
    
    ####################
    # Scale Complexity #
    ####################
    
    try:
        fp = f'./street-network-{ID}.png'
        ox.plot_graph(G,bgcolor='white',node_color='black',edge_color='black',show=False,close=True,dpi=150,save=True,filepath=fp)
        I = imageio.imread(fp, as_gray="True")/255.0
        ! rm $fp # comment if you want to save the plot
        gdf.loc[ID,'fractal-dimension'] = fractal_dimension(I)
    except:
        pass
  
    try:
        gdf.loc[ID,'compactness-area'] = pp_compactness(polygon)
    except:
        pass
    
    try:
        gdf.loc[ID,'diameter-periphery'] = nx.diameter(G.to_undirected())
    except:
        pass
    
    #######################################
    # Spatial Complexity and Connectivity #
    #######################################
    
    try:
        # get 'shannon_entropy-street_orientation_order'
        bearings = get_bearings(G, ID)
        count = count_and_merge(36, bearings)
        street_orientation_order = get_orientation_order(count)
        gdf.loc[ID,'shannon_entropy-street_orientation_order'] = street_orientation_order
        #print(street_orientation_order)
    except:
        pass
    
    try:
        # get basic stats from network
        basic = ox.stats.basic_stats(G, area=gdf.loc[ID,"area_m^2"]) # basic stats
    
        # allocate stats
        gdf.loc[ID,'avg_streets_per_node'] = basic['streets_per_node_avg']
        gdf.loc[ID,'avg_proportion_streets_per_node'] = np.mean(list(basic['streets_per_node_proportions'].values()))
        gdf.loc[ID,'avg_street_length'] = basic['street_length_avg']
        gdf.loc[ID,'intersection_density'] = basic['node_density_km']
        gdf.loc[ID,'street_density'] = basic['street_density_km']
        gdf.loc[ID,'avg_circuity'] = basic['circuity_avg']
        gdf.loc[ID,'avg_node_connectivity'] = np.mean(nx.node_connectivity(G))
        gdf.loc[ID,'avg_PageRank'] = np.mean(list(nx.pagerank(G).values()))
    except:
        pass
    
    try:
        # multiple centrality assessment
        streets_graph = ox.projection.project_graph(G)
        edges = ox.graph_to_gdfs(ox.get_undirected(streets_graph), nodes=False, edges=True,node_geometry=False, fill_edge_geometry=True)
        primal = momepy.gdf_to_nx(edges, approach='primal')
        primal = momepy.closeness_centrality(primal, radius=gdf.loc[ID,'diameter-periphery']*(1/4), name='closeness_local', distance='mm_len', weight='mm_len')
        primal = momepy.closeness_centrality(primal, name='closeness_global', weight='mm_len')
        primal = momepy.betweenness_centrality(primal, name='betweenness_metric_n', mode='nodes', weight='mm_len')
        primal = momepy.straightness_centrality(primal)
        nodes = momepy.nx_to_gdf(primal, lines=False)

        gdf.loc[ID,'avg_betweenness_centrality'] = np.mean(nodes['betweenness_metric_n'])
        gdf.loc[ID,'avg_local_closeness_centrality'] = np.mean(nodes['closeness_local'])
        gdf.loc[ID,'avg_global_closeness_centrality'] = np.mean(nodes['closeness_global'])
        gdf.loc[ID,'avg_straightness_centrality'] = np.mean(nodes['straightness'])
    except:
        pass
    
    
    ###############################
    # Built Complexity/Morphology #
    ###############################
    
    try:
        buildings_gdf = ox.geometries_from_polygon(polygon, tags={'building':True})
        buildings_gdf_projected = ox.project_gdf(buildings_gdf)
        buildings_gdf_projected = buildings_gdf_projected.reset_index()
        buildings_gdf_projected = buildings_gdf_projected[buildings_gdf_projected.geom_type.isin(['Polygon', 'MultiPolygon'])]

        buildings = momepy.preprocess(buildings_gdf_projected, size=30, compactness=True, islands=True)
        buildings['uID'] = momepy.unique_id(buildings)

        limit = momepy.buffered_limit(buildings)

        tess = momepy.Tessellation(buildings, unique_id='uID', limit=limit)
        tessellation = tess.tessellation
        
    except:
        pass
    
    try:
        blg_area = momepy.Area(buildings)
        buildings['area'] = blg_area.series
        gdf.loc[ID,'avg_building_area'] = buildings['area'].mean()

        tes_area = momepy.Area(tessellation)
        tessellation['area'] = tes_area.series
        gdf.loc[ID,'avg_tesselation_area'] = tessellation['area'].mean()
    
    except:
        pass
    
    try:
        # try to fix heights
        buildings['height'] = buildings['height'].fillna(0).apply(clean_heights)
        if buildings['height'].mean() == 0:
            pass
        else:
            gdf.loc[ID,'avg_building_height'] = buildings['height'].mean()

        blg_volume = momepy.Volume(buildings, heights='height')
        buildings['volume'] = blg_volume.series
        if buildings['volume'].mean() == 0:
            pass
        else:
            gdf.loc[ID,'avg_building_volume'] = buildings['volume'].mean()
    except:
        pass
    
    
    try:    
        buildings['orientation'] = momepy.Orientation(buildings).series
        gdf.loc[ID,'avg_building_orientation'] = buildings['orientation'].mean()

        tessellation['orientation'] = momepy.Orientation(tessellation).series
        gdf.loc[ID,'avg_tessellation_orientation'] = tessellation['orientation'].mean() 

        blg_cell_align = momepy.CellAlignment(buildings, tessellation,'orientation', 'orientation','uID', 'uID')
        buildings['cell_align'] = blg_cell_align.series
        gdf.loc[ID,'avg_building_cell_alignment'] = buildings['cell_align'].mean()
    except:
        pass
    
    try:
        edges['networkID'] = momepy.unique_id(edges)
        buildings['networkID'] = momepy.get_network_id(buildings, edges, 'networkID')
        buildings_net = buildings.loc[buildings.networkID >= 0]
        str_align = momepy.StreetAlignment(buildings_net, edges,'orientation', 'networkID','networkID')
        buildings_net['str_align'] = str_align.series
        gdf.loc[ID,'avg_street_alignment'] = buildings_net['str_align'].mean()
    except:
        pass
        
    try:
        profile = momepy.StreetProfile(edges, buildings, heights='height')
        edges['widths'] = profile.w
        edges['width_deviations'] = profile.wd
        edges['openness'] = profile.o
        edges['heights'] = profile.h
        edges['heights_deviations'] = profile.hd
        edges['profile'] = profile.p
    
        gdf.loc[ID,'avg_width-street_profile'] = edges['widths'].mean()
        gdf.loc[ID,'avg_width_deviations-street_profile'] = edges['widths'].mean()
        gdf.loc[ID,'avg_openness-street_profile'] = edges['widths'].mean()
        gdf.loc[ID,'avg_heights-street_profile'] = edges['widths'].mean()
        gdf.loc[ID,'avg_heights_deviations-street_profile'] = edges['widths'].mean()
        gdf.loc[ID,'avg_profile-street_profile'] = edges['widths'].mean()
        
    except:
        pass
    
    try:
        buildings['compactness'] = buildings['geometry'].apply(lambda x: pp_compactness(x))
        gdf.loc[ID,'avg_building_compactness'] = buildings['compactness'].mean()
    except:
        pass
    
    #################################
    # Infrastructure variables/KPIs #
    ################################# 
    
    try:
        gdf.loc[ID,'total_area'] = gdf.loc[ID,"area_m^2"]
        gdf.loc[ID,'total_built_area'] = buildings['area'].sum()
        gdf.loc[ID,'total_street_length'] = basic['street_length_total']
        
    except:
        pass
    
gdf['UID'] = gdf.index 

Run 1 out of 1: ID = 0


# Report data availability

In [7]:
vars_of_interest = [
    'avg_betweenness_centrality',
    'fractal-dimension',
    'shannon_entropy-street_orientation_order',
    'avg_street_length',
    'avg_building_area',
    'avg_building_compactness']

for variable in vars_of_interest:
    if gdf[variable].isna().all():
        print(f"{variable:<40} -> missing")
    else:
        print(f"{variable:<40} -> available")

avg_betweenness_centrality               -> missing
fractal-dimension                        -> missing
shannon_entropy-street_orientation_order -> available
avg_street_length                        -> available
avg_building_area                        -> missing
avg_building_compactness                 -> missing


# Save file

In [8]:
out_file = data_folder / "2_morphometrics" / (city + " - morpho.gpkg")
print("Saving:", out_file)
gdf.to_file(out_file, driver='GPKG')

Saving: ../data/morphometrics/Los Angeles - morpho.gpkg
